In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, LSTM, GRU, Embedding, SimpleRNN, Dropout
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    HistGradientBoostingClassifier,
    BaggingClassifier
)
from tqdm import tqdm

from sklearn.linear_model import (
    LogisticRegression,
    RidgeClassifier,
    SGDClassifier,
    PassiveAggressiveClassifier,  # Add this line
    Perceptron,
)


from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    HistGradientBoostingClassifier,
    BaggingClassifier,
    ExtraTreesClassifier  # Make sure this is included
)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import VarianceThreshold, RFECV, SelectKBest, f_classif
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


In [2]:
data = pd.read_csv('/content/train.csv')
data.head(5)

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,...,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,sampen,higuci,datasetId,condition
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,11.001565,...,15.522602,0.421047,1.514737,3686.666157,65.018055,0.015380,2.139754,1.163485,2,0
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,9.170129,...,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,2.174499,1.084711,2,1
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,11.533417,...,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,2.135350,1.176315,2,1
3,881.757864,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,11.119476,...,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,2.178341,1.179688,2,0
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,13.590641,...,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,2.221121,1.249612,2,0


In [3]:
data.isna().sum()


,0
MEAN_RR,0
MEDIAN_RR,0
SDRR,0
RMSSD,0
SDSD,0
SDRR_RMSSD,0
HR,0
pNN25,0
pNN50,0
SD1,0


In [4]:
X= data.drop(
    columns=['condition'], axis =1
)
y = data.condition

In [5]:
X

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,...,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,sampen,higuci,datasetId
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,11.001565,...,98.485263,15.522602,0.421047,1.514737,3686.666157,65.018055,0.015380,2.139754,1.163485,2
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,9.170129,...,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,2.174499,1.084711,2
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,11.533417,...,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,2.135350,1.176315,2
3,881.757864,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,11.119476,...,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,2.178341,1.179688,2
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,13.590641,...,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,2.221121,1.249612,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369284,721.396910,721.533965,36.377559,9.609941,9.609936,3.785409,83.384647,0.933333,0.000000,6.797519,...,89.119683,38.227175,5.886975,10.880317,649.351702,8.190908,0.122087,2.096850,1.314910,2
369285,984.266492,978.622945,74.918433,13.762274,13.761705,5.443754,61.314243,6.333333,0.200000,9.734243,...,99.642453,2.573834,0.112964,0.357547,2278.456746,278.683532,0.003588,2.131264,1.175975,2
369286,1025.499743,1024.968400,95.309200,23.051254,23.050395,4.134664,59.028594,30.400000,2.066667,16.304530,...,99.819930,4.287216,0.098983,0.180070,4331.259564,554.340761,0.001804,2.215643,1.140232,2
369287,798.123167,803.559610,78.449897,16.736686,16.736657,4.687302,75.978628,10.000000,1.466667,11.838553,...,96.477005,46.636158,1.591647,3.522995,2930.056129,27.384942,0.036516,2.091628,1.181937,2


In [6]:
y

,condition
0,0
1,1
2,1
3,0
4,0
...,...
369284,0
369285,0
369286,0
369287,0


In [7]:
rfc = RandomForestClassifier(random_state=1)
rfecv = RFECV(estimator=rfc, step =1, cv =5, scoring = 'accuracy')

rfecv.fit(X.select_dtypes(exclude = "object"), y)

KeyboardInterrupt: 

In [ ]:
print('Optimal Number of features is : {}'.format(rfecv.n_features_))


In [ ]:
X.drop(X.columns[np.where(rfecv.support_ == False)[0]], axis =1 ,inplace=True)


In [7]:
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(X.select_dtypes(exclude = "object"))

constant_columns = [column for column in X.select_dtypes(exclude = "object").columns
                    if column not in X.select_dtypes(exclude = "object").columns[var_thres.get_support()]]

print(len(constant_columns))

1


In [8]:
constant_columns

['datasetId']

In [9]:
X.drop(['datasetId'], axis =1, inplace = True)

In [10]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [11]:
corr_features = correlation(X, 0.8)
len(set(corr_features))

19

In [12]:
X.drop(corr_features,axis=1,inplace=True)

In [13]:
X

,MEAN_RR,SDRR,RMSSD,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_RMSSD_REL_RR,VLF,VLF_PCT,LF_NU,LF_HF,sampen,higuci
0,885.157845,140.972741,15.554505,0.533333,-0.856554,0.335218,-0.000203,-0.000179,2.143342,2661.894136,72.203287,98.485263,65.018055,2.139754,1.163485
1,939.425371,81.317742,12.964439,0.000000,-0.408190,-0.155286,-0.000059,0.000611,2.930855,2314.265450,76.975728,99.695397,327.296635,2.174499,1.084711
2,898.186047,84.497236,16.305279,0.200000,0.351789,-0.656813,-0.000011,-0.000263,2.127053,1373.887112,51.152225,98.950472,94.280910,2.135350,1.176315
3,881.757864,90.370537,15.720468,0.133333,-0.504947,-0.386138,0.000112,0.000494,2.050988,2410.357408,70.180308,98.224706,55.328701,2.178341,1.179688
4,809.625331,62.766242,19.213819,0.200000,-0.548408,-0.154252,-0.000100,-0.002736,1.816544,1151.177330,43.918366,96.720007,29.487873,2.221121,1.249612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369284,721.396910,36.377559,9.609941,0.000000,1.007321,0.075965,0.000015,0.000082,1.529068,298.009244,45.893349,89.119683,8.190908,2.096850,1.314910
369285,984.266492,74.918433,13.762274,0.200000,0.248964,0.167716,-0.000110,0.000046,2.218313,1558.597883,68.405858,99.642453,278.683532,2.131264,1.175975
369286,1025.499743,95.309200,23.051254,2.066667,-0.429686,-0.075321,-0.000216,-0.002236,2.391601,1950.393956,45.030641,99.819930,554.340761,2.215643,1.140232
369287,798.123167,78.449897,16.736686,1.466667,0.600389,-0.634762,0.000024,-0.001354,1.894304,1606.291482,54.821185,96.477005,27.384942,2.091628,1.181937


In [14]:
y

,condition
0,0
1,1
2,1
3,0
4,0
...,...
369284,0
369285,0
369286,0
369287,0


In [15]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 20)


In [16]:
x_train.shape, x_test.shape


((295431, 15), (73858, 15))

In [22]:
# determine the mutual information
mutual_info = mutual_info_classif(x_train.select_dtypes(exclude="object"), y_train)
mutual_info

array([0.20743453, 0.14796366, 0.12030923, 0.0586507 , 0.06580947,
       0.05581813, 0.00444954, 0.90108774, 0.10264371, 0.07175213,
       0.06355842, 0.07662289, 0.07662732, 0.05370368, 0.05486152])

In [24]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = x_train.select_dtypes(exclude="object").columns
mutual_info.sort_values(ascending=False)

,0
MEDIAN_REL_RR,0.901088
MEAN_RR,0.207435
SDRR,0.147964
RMSSD,0.120309
SDRR_RMSSD_REL_RR,0.102644
LF_HF,0.076627
LF_NU,0.076623
VLF,0.071752
KURT,0.065809
VLF_PCT,0.063558


In [19]:
num_features = X.select_dtypes(exclude = "object")
cat_features = X.select_dtypes(include = "object")

In [20]:
y.value_counts()

,count
condition,
0,200082
1,105150
2,64057


In [21]:
num_features


,MEAN_RR,SDRR,RMSSD,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_RMSSD_REL_RR,VLF,VLF_PCT,LF_NU,LF_HF,sampen,higuci
0,885.157845,140.972741,15.554505,0.533333,-0.856554,0.335218,-0.000203,-0.000179,2.143342,2661.894136,72.203287,98.485263,65.018055,2.139754,1.163485
1,939.425371,81.317742,12.964439,0.000000,-0.408190,-0.155286,-0.000059,0.000611,2.930855,2314.265450,76.975728,99.695397,327.296635,2.174499,1.084711
2,898.186047,84.497236,16.305279,0.200000,0.351789,-0.656813,-0.000011,-0.000263,2.127053,1373.887112,51.152225,98.950472,94.280910,2.135350,1.176315
3,881.757864,90.370537,15.720468,0.133333,-0.504947,-0.386138,0.000112,0.000494,2.050988,2410.357408,70.180308,98.224706,55.328701,2.178341,1.179688
4,809.625331,62.766242,19.213819,0.200000,-0.548408,-0.154252,-0.000100,-0.002736,1.816544,1151.177330,43.918366,96.720007,29.487873,2.221121,1.249612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369284,721.396910,36.377559,9.609941,0.000000,1.007321,0.075965,0.000015,0.000082,1.529068,298.009244,45.893349,89.119683,8.190908,2.096850,1.314910
369285,984.266492,74.918433,13.762274,0.200000,0.248964,0.167716,-0.000110,0.000046,2.218313,1558.597883,68.405858,99.642453,278.683532,2.131264,1.175975
369286,1025.499743,95.309200,23.051254,2.066667,-0.429686,-0.075321,-0.000216,-0.002236,2.391601,1950.393956,45.030641,99.819930,554.340761,2.215643,1.140232
369287,798.123167,78.449897,16.736686,1.466667,0.600389,-0.634762,0.000024,-0.001354,1.894304,1606.291482,54.821185,96.477005,27.384942,2.091628,1.181937


In [22]:
cat_features

""
0
1
2
3
4
...
369284
369285
369286
369287


In [23]:
from sklearn.compose import ColumnTransformer

step1 = ColumnTransformer(
    transformers=[
        ('col_tnf', OneHotEncoder( drop='first', handle_unknown='ignore'),cat_features.columns),
        ('num_tnf', StandardScaler(),num_features.columns )
    ],
    remainder='passthrough'
)

In [ ]:
step1

In [31]:
warnings.filterwarnings("ignore")

def evaluate_model(model, x_train, y_train, x_test, y_test):
    # Preprocess 'previous_year_rating' column


    pipe = Pipeline([
        ('Transform', step1),  # You need to define the 'step1' transformer
        ('model', model),
    ])

    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)

    # Use appropriate classification metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred,  average="macro")
    f1 = f1_score(y_test, y_pred,  average="macro")

    # Calculate the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    return accuracy, precision, recall, f1, cm


# Initialize different classification models
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest Classifier': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier(),
    'AdaBoost Classifier': AdaBoostClassifier(),
    'Bagging Classifier': BaggingClassifier(),
    'Extra Trees Classifier': ExtraTreesClassifier(),
    'Support Vector Classifier': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree Classifier': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB(),
    'HistGradientBoosting Classifier': HistGradientBoostingClassifier(),
    'SGD Classifier': SGDClassifier(),
    'Passive Aggressive Classifier': PassiveAggressiveClassifier(),
    'Ridge Classifier': RidgeClassifier(),
    'MLP Neural Network': MLPClassifier(hidden_layer_sizes=(10,), max_iter=100),

}
# Create a list to store the results
results = []

# Evaluate and print classification metrics for different models
for model_name, model in tqdm(models.items()):
    accuracy, precision, recall, f1, cm = evaluate_model(model, x_train, y_train, x_test, y_test)
    results.append([model_name, accuracy, precision, recall, f1, cm])

# Print the results in a tabular form
headers = ["Model", "Accuracy", "Precision", "Recall", "F1 Score", "Confusion Matrix"]
print(tabulate(results, headers, tablefmt="grid"))

max_accuracy_index = np.argmax([result[1] for result in results])
best_model_name = results[max_accuracy_index][0]
print(f"\nThe model with the highest accuracy is: {best_model_name} (Accuracy: {results[max_accuracy_index][1]:.4f})")

100%|██████████| 15/15 [39:56<00:00, 159.80s/it]


NameError: name 'tabulate' is not defined

In [33]:
from tabulate import tabulate

# Print the results in a tabular form
headers = ["Model", "Accuracy", "Precision", "Recall", "F1 Score", "Confusion Matrix"]
print(tabulate(results, headers, tablefmt="grid"))

+---------------------------------+------------+-------------+----------+------------+-----------------------+
| Model                           |   Accuracy |   Precision |   Recall |   F1 Score | Confusion Matrix      |
+=================================+============+=============+==========+============+=======================+
| Logistic Regression             |   0.610875 |    0.577902 | 0.489239 |   0.502232 | [[34454  4318  1266]  |
|                                 |            |             |          |            |  [12013  7375  1415]  |
|                                 |            |             |          |            |  [ 8024  1704  3289]] |
+---------------------------------+------------+-------------+----------+------------+-----------------------+
| Random Forest Classifier        |   1        |    1        | 1        |   1        | [[40038     0     0]  |
|                                 |            |             |          |            |  [    0 20803     0]  |
|

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc, precision_recall_curve
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from sklearn.pipeline import Pipeline

warnings.filterwarnings("ignore")


# Define your class labels (replace with your actual class labels)
class_labels = [0, 1, 2]

# Define your step1 transformer (replace with your actual transformer)
# step1 = <your_step1_transformer>

def evaluate_model(model, x_train, y_train, x_test, y_test):
    pipe = Pipeline([
        ('Transform', step1),  # You need to define the 'step1' transformer if required
        ('model', model),
    ])

    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)

    # Convert string labels to binary labels (if necessary)
    label_mapping = {label: idx for idx, label in enumerate(class_labels)}
    y_test_binary = np.array([label_mapping[label] for label in y_test])
    y_pred_binary = np.array([label_mapping[label] for label in y_pred])

    # Use appropriate classification metrics
    accuracy = accuracy_score(y_test_binary, y_pred_binary)
    precision = precision_score(y_test_binary, y_pred_binary, average="macro")
    recall = recall_score(y_test_binary, y_pred_binary, average="macro")
    f1 = f1_score(y_test_binary, y_pred_binary, average="macro")

    # Calculate the confusion matrix with class labels
    cm = confusion_matrix(y_test_binary, y_pred_binary)

    # Normalize confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # Normalize each row by the sum of that row

    # Calculate ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_test_binary, y_pred_binary)
    roc_auc = auc(fpr, tpr)

    # Calculate Precision-Recall curve
    precision_curve, recall_curve, _ = precision_recall_curve(y_test_binary, y_pred_binary)

    return accuracy, precision, recall, f1, cm, cm_normalized, fpr, tpr, roc_auc, precision_curve, recall_curve

# Initialize different classification models
models = {
    'Support Vector Classifier': SVC(),
}

# Create a list to store the results
results = []

# Evaluate and print classification metrics for different models
for model_name, model in models.items():
    accuracy, precision, recall, f1, cm, cm_normalized, fpr, tpr, roc_auc, precision_curve, recall_curve = evaluate_model(model, x_train, y_train, x_test, y_test)
    results.append([model_name, accuracy, precision, recall, f1, cm, cm_normalized, roc_auc, precision_curve, recall_curve])

# Print the results in a tabular form
headers = ["Model", "Accuracy", "Precision", "Recall", "F1 Score"]
print(tabulate(results, headers, tablefmt="grid"))

max_accuracy_index = np.argmax([result[1] for result in results])
best_model_name = results[max_accuracy_index][0]
print(f"\nThe model with the highest accuracy is: {best_model_name} (Accuracy: {results[max_accuracy_index][1]:.4f})")

# Plot confusion matrix with class labels and show labels
plt.figure(figsize=(8, 6))
sns.heatmap(results[max_accuracy_index][5], annot=True, fmt=".2f", cmap="Blues", cbar=False, vmin=0, vmax=1,
            annot_kws={"color": "red"})  # Normalized confusion matrix
plt.title(f"Normalized Confusion Matrix for {best_model_name}")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.xticks(np.arange(len(class_labels)) + 0.5, class_labels)
plt.yticks(np.arange(len(class_labels)) + 0.5, class_labels)
plt.show()

# Plot ROC curve
plt.figure(figsize=(8, 8))
plt.plot(results[max_accuracy_index][6], results[max_accuracy_index][7], color='darkorange', lw=2,
         label='ROC curve (AUC = {:.2f})'.format(results[max_accuracy_index][8]))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Plot Precision-Recall curve
plt.figure(figsize=(8, 8))
plt.plot(results[max_accuracy_index][9], results[max_accuracy_index][10], color='darkorange', lw=2,
         label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='upper right')
plt.show()


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Define the SVC model
model = SVC()

# Define the pipeline
pipe = Pipeline([
    ('Transform', step1),  # Ensure 'step1' is defined correctly
    ('model', model),
])

# Fit the pipeline on the training data
pipe.fit(x_train, y_train)

# Predict using the pipeline (applies transformations + prediction)
y_train_pred = pipe.predict(x_train)
y_test_pred = pipe.predict(x_test)

# Calculate metrics on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='macro')
train_recall = recall_score(y_train, y_train_pred, average='macro')
train_f1 = f1_score(y_train, y_train_pred, average='macro')

# Calculate metrics on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='macro')
test_recall = recall_score(y_test, y_test_pred, average='macro')
test_f1 = f1_score(y_test, y_test_pred, average='macro')

# Create a DataFrame to display metrics
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Train Set': [train_accuracy, train_precision, train_recall, train_f1],
    'Test Set': [test_accuracy, test_precision, test_recall, test_f1]
})

print(metrics_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a bar plot to visualize the metrics comparison between Train and Test sets
metrics_df.set_index('Metric', inplace=True)

plt.figure(figsize=(8, 6))
sns.barplot(x=metrics_df.index, y='Train Set', data=metrics_df, color='lightblue', label='Train Set')
sns.barplot(x=metrics_df.index, y='Test Set', data=metrics_df, color='lightgreen', label='Test Set')

# Add labels and title
plt.title('Model Performance Metrics: Train vs Test')
plt.ylabel('Score')
plt.xlabel('Metrics')
plt.legend(title="Dataset")

# Display the plot
plt.show()


# Creating ANN Model


In [ ]:
model = Sequential()
model.add(Dense(50, activation='relu',kernel_initializer = 'he_normal',input_shape=(x_train.shape[1],)))
model.add(Dense(20, activation='relu',kernel_initializer = 'he_normal'))
model.add(Dense(10, activation='relu',kernel_initializer = 'he_normal'))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(x_train,y_train, validation_data = (x_test,y_test), epochs = 50, batch_size = 1024, verbose =1)


In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,7))
